In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#전처리
from sklearn.preprocessing import LabelEncoder, StandardScaler

#데이터분리
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV

#모델링
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier

#성과 평가 지표
from sklearn.metrics import accuracy_score, mean_squared_error

In [2]:
train=pd.read_csv('train.csv', index_col=0)
test=pd.read_csv('test_x.csv', index_col=0)
submission=pd.read_csv('sample_submission.csv', index_col=0)  
print(train.shape)
print(test.shape)
print(submission.shape)

(45532, 77)
(11383, 76)
(11383, 1)


In [3]:
#타겟인 'voted'변수를 train_y로 만들어줍니다
train_x = train.drop('voted', axis = 1)
train_y = train['voted']

In [4]:
#빠른 제출 절차를 확인하기 위해 '문자열' 변수들을 '이산형'으로 변환해줍니다.
#age_group, gender, race, religion
train_x['age_group']=1
train_x['gender']=1
train_x['race']=1
train_x['religion']=1

test['age_group']=1
test['gender']=1
test['race']=1
test['religion']=1

In [7]:
#질문만 일단 제거해본다.
drop_val = ['QaA', 'QbA',  'QcA','QdA',  
            'QeA','QfA',  'QgA',  'QhA',  
            'QiA',  'QjA', 'QkA', 'QlA', 
            'QmA',  'QnA',  'QoA', 'QpA', 
            'QqA',  'QrA',  'QsA',  'QtA',]
train = train_x.drop(drop_val, axis = 1)
test = test.drop(drop_val, axis = 1)
train.head()

,QaE,QbE,QcE,QdE,QeE,QfE,QgE,QhE,QiE,QjE,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
index,,,,,,,,,,,,,,,,,,,,,
0,363,1370,997,1024,1577,539,586,1095,1142,1287,...,0,1,0,1,1,0,1,0,1,1
1,647,1313,3387,2969,4320,2190,826,4082,1867,1264,...,1,1,0,1,1,0,1,0,1,1
2,1623,1480,1021,3374,1333,531,1167,1016,2653,1569,...,1,1,0,1,1,1,1,0,1,1
3,504,2311,992,3245,357,1519,159,2275,2809,5614,...,0,0,0,0,1,0,1,0,1,1
4,927,707,556,1062,1014,628,991,1259,1153,1388,...,1,1,1,1,1,0,1,1,1,1


In [8]:
train_dummies = pd.get_dummies(train)
test_dummies = pd.get_dummies(test)

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(train_dummies)
train_dummies_1 = scaler.transform(train_dummies)
train_dummies_2 = pd.DataFrame(data=train_dummies_1,columns = train_dummies.columns)
train_dummies_2

,QaE,QbE,QcE,QdE,QeE,QfE,QgE,QhE,QiE,QjE,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,-0.044538,-0.024458,-0.054279,-0.042726,-0.019290,-0.017240,-0.071605,-0.006950,-0.025632,-0.023827,...,-2.572274,0.341201,-0.699491,0.255100,0.169115,-0.504409,0.238676,-0.808347,0.234849,0.280538
1,-0.022818,-0.026159,0.211940,0.135347,0.144888,0.004460,-0.047005,-0.001001,-0.019282,-0.024252,...,0.388761,0.341201,-0.699491,0.255100,0.169115,-0.504409,0.238676,-0.808347,0.234849,0.280538
2,0.051825,-0.021175,-0.051606,0.172427,-0.033895,-0.017345,-0.012054,-0.007108,-0.012397,-0.018616,...,0.388761,0.341201,-0.699491,0.255100,0.169115,1.982517,0.238676,-0.808347,0.234849,0.280538
3,-0.033754,0.003623,-0.054836,0.160616,-0.092312,-0.004359,-0.115370,-0.004600,-0.011031,0.056134,...,-2.572274,-2.930823,-0.699491,-3.920031,0.169115,-0.504409,0.238676,-0.808347,0.234849,0.280538
4,-0.001404,-0.044243,-0.103402,-0.039247,-0.052988,-0.016070,-0.030094,-0.006624,-0.025535,-0.021960,...,0.388761,0.341201,1.429611,0.255100,0.169115,-0.504409,0.238676,1.237093,0.234849,0.280538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45527,0.008003,-0.046869,-0.128798,-0.110384,-0.077647,-0.020815,-0.099381,-0.008170,-0.026735,-0.042473,...,0.388761,0.341201,-0.699491,0.255100,0.169115,1.982517,0.238676,-0.808347,0.234849,0.280538
45528,-0.027866,-0.024965,-0.035677,-0.063417,-0.013125,-0.016964,-0.066070,-0.006313,0.003727,-0.021887,...,0.388761,0.341201,-0.699491,0.255100,0.169115,1.982517,0.238676,-0.808347,0.234849,0.280538
45529,-0.026948,-0.039767,-0.048710,0.002227,-0.012527,-0.007855,-0.019537,-0.006576,-0.016286,-0.015012,...,0.388761,0.341201,-0.699491,0.255100,0.169115,-0.504409,0.238676,-0.808347,0.234849,0.280538
45530,-0.015170,-0.025622,-0.065975,-0.019196,-0.034194,-0.016451,-0.065045,-0.006285,-0.020324,-0.033381,...,0.388761,0.341201,-0.699491,0.255100,0.169115,1.982517,0.238676,-0.808347,0.234849,0.280538


In [16]:
scaler = StandardScaler()
scaler.fit(test_dummies)
test_dummies_1 = scaler.transform(test_dummies)
test_dummies_2 = pd.DataFrame(data=test_dummies_1,columns = test_dummies.columns)
test_dummies_2

,QaE,QbE,QcE,QdE,QeE,QfE,QgE,QhE,QiE,QjE,...,wr_04,wr_05,wr_06,wr_07,wr_08,wr_09,wr_10,wr_11,wr_12,wr_13
0,-0.036553,0.011065,-0.000018,0.169896,0.038951,0.200620,0.107099,-0.117878,-0.023081,0.172670,...,-2.629801,0.339916,-0.699944,-3.900654,0.168977,-0.496071,0.238574,-0.809907,0.227040,0.284797
1,-0.127165,-0.008293,-0.009034,-0.038935,-0.046033,-0.069040,-0.045853,0.077041,-0.017749,-0.023912,...,-2.629801,-2.941903,-0.699944,-3.900654,-5.917952,-0.496071,-4.191579,-0.809907,-4.404502,-3.511273
2,-0.132879,0.002570,-0.012183,-0.049645,-0.035349,-0.052016,-0.035798,-0.159042,-0.009414,-0.051283,...,-2.629801,0.339916,-0.699944,0.256367,0.168977,-0.496071,0.238574,-0.809907,0.227040,0.284797
3,-0.063900,-0.025949,-0.009378,0.035108,0.037919,-0.071877,-0.038693,0.191857,-0.024814,-0.034028,...,0.380257,0.339916,1.428686,0.256367,0.168977,2.015840,0.238574,1.234709,0.227040,0.284797
4,-0.133288,-0.022171,-0.011111,0.004787,-0.059691,-0.089217,-0.058040,-0.228910,-0.024432,-0.042269,...,0.380257,0.339916,-0.699944,0.256367,0.168977,-0.496071,0.238574,1.234709,0.227040,0.284797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11378,-0.162675,-0.025636,-0.011866,-0.052439,-0.048461,-0.086590,-0.057177,-0.090731,-0.024265,-0.020412,...,0.380257,0.339916,-0.699944,0.256367,0.168977,-0.496071,0.238574,-0.809907,0.227040,0.284797
11379,-0.208797,-0.035657,-0.012918,-0.031949,-0.044636,-0.089112,-0.044025,-0.083166,-0.029422,-0.038769,...,0.380257,0.339916,1.428686,0.256367,0.168977,2.015840,0.238574,1.234709,0.227040,0.284797
11380,-0.081043,-0.030862,-0.011422,-0.023309,-0.065518,-0.093736,-0.054536,-0.087616,-0.028914,-0.032374,...,0.380257,0.339916,-0.699944,0.256367,0.168977,-0.496071,0.238574,-0.809907,0.227040,0.284797
11381,-0.116961,-0.005592,0.000044,-0.046592,-0.043908,-0.077447,-0.002080,-0.120325,-0.026927,-0.048058,...,0.380257,-2.941903,-0.699944,0.256367,0.168977,-0.496071,0.238574,1.234709,0.227040,-3.511273


In [17]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

# 개별 기반 모델에서 최종 메타 모델이 사용할 학습 및 테스트용 데이터를 생성하기 위한 함수. 
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds ):
    # 지정된 n_folds값으로 KFold 생성.
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=0)
    #추후에 메타 모델이 사용할 학습 데이터 반환을 위한 넘파이 배열 초기화 
    train_fold_pred = np.zeros((X_train_n.shape[0] ,1 ))
    test_pred = np.zeros((X_test_n.shape[0],n_folds))
    print(model.__class__.__name__ , ' model 시작 ')
    
    for folder_counter , (train_index, valid_index) in enumerate(kf.split(X_train_n)):
        #입력된 학습 데이터에서 기반 모델이 학습/예측할 폴드 데이터 셋 추출 
        print('\t 폴드 세트: ',folder_counter,' 시작 ')
        X_tr = X_train_n.iloc[train_index] 
        y_tr = y_train_n.iloc[train_index] 
        X_te = X_train_n.iloc[valid_index]  
        
        #폴드 세트 내부에서 다시 만들어진 학습 데이터로 기반 모델의 학습 수행.
        model.fit(X_tr , y_tr)       
        #폴드 세트 내부에서 다시 만들어진 검증 데이터로 기반 모델 예측 후 데이터 저장.
        train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
        #입력된 원본 테스트 데이터를 폴드 세트내 학습된 기반 모델에서 예측 후 데이터 저장. 
        test_pred[:, folder_counter] = model.predict(X_test_n)
            
    # 폴드 세트 내에서 원본 테스트 데이터를 예측한 데이터를 평균하여 테스트 데이터로 생성 
    test_pred_mean = np.mean(test_pred, axis=1).reshape(-1,1)    
    
    #train_fold_pred는 최종 메타 모델이 사용하는 학습 데이터, test_pred_mean은 테스트 데이터
    return train_fold_pred , test_pred_mean

In [18]:
knn_clf  = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)


In [20]:

knn_train, knn_test = get_stacking_base_datasets(knn_clf, train_dummies_2, train_y, test_dummies_2, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, train_dummies_2, train_y, test_dummies_2, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, train_dummies_2, train_y, test_dummies_2,  7)    
ada_train, ada_test = get_stacking_base_datasets(ada_clf, train_dummies_2, train_y, test_dummies_2, 7)

KNeighborsClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
RandomForestClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
DecisionTreeClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 
AdaBoostClassifier  model 시작 
	 폴드 세트:  0  시작 
	 폴드 세트:  1  시작 
	 폴드 세트:  2  시작 
	 폴드 세트:  3  시작 
	 폴드 세트:  4  시작 
	 폴드 세트:  5  시작 
	 폴드 세트:  6  시작 


In [22]:
# lgbm으로 
import lightgbm as lgbm
lgbm_clf = lgbm.LGBMClassifier(n_estimators=200)

In [23]:
from sklearn.model_selection import GridSearchCV

parameters = {'min_data_in_leaf':[15,20,25],'max_depth':[64,74,84]}
gridcv = GridSearchCV(lgbm_clf, param_grid=parameters, cv=3)
gridcv.fit(train_dummies_2, train_y, early_stopping_rounds=30, eval_metric="auc",
          eval_set = (train_dummies_2, train_y))

print('GridSearchCV 최적 파라미터:',gridcv.best_params_)

[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[1]	valid_0's auc: 0.734118	valid_0's binary_logloss: 0.671901
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.737149	valid_0's binary_logloss: 0.658065
[3]	valid_0's auc: 0.739634	valid_0's binary_logloss: 0.646622
[4]	valid_0's auc: 0.741584	valid_0's binary_logloss: 0.636976
[5]	valid_0's auc: 0.742672	valid_0's binary_logloss: 0.628898
[6]	valid_0's auc: 0.743953	valid_0's binary_logloss: 0.621978
[7]	valid_0's auc: 0.744388	valid_0's binary_logloss: 0.616126
[8]	valid_0's auc: 0.745471	valid_0's binary_logloss: 0.611139
[9]	valid_0's auc: 0.74627	valid_0's binary_logloss: 0.6068
[10]	valid_0's auc: 0.747919	valid_0's binary_logloss: 0.602949
[11]	valid_0's auc: 0.748561	valid_0's binary_logloss: 0.599724
[12]	valid_0's auc: 0.7

[138]	valid_0's auc: 0.822845	valid_0's binary_logloss: 0.522379
[139]	valid_0's auc: 0.823273	valid_0's binary_logloss: 0.522018
[140]	valid_0's auc: 0.823894	valid_0's binary_logloss: 0.52153
[141]	valid_0's auc: 0.824224	valid_0's binary_logloss: 0.52126
[142]	valid_0's auc: 0.824556	valid_0's binary_logloss: 0.520915
[143]	valid_0's auc: 0.825133	valid_0's binary_logloss: 0.520489
[144]	valid_0's auc: 0.825341	valid_0's binary_logloss: 0.520178
[145]	valid_0's auc: 0.825922	valid_0's binary_logloss: 0.519711
[146]	valid_0's auc: 0.826308	valid_0's binary_logloss: 0.519393
[147]	valid_0's auc: 0.826644	valid_0's binary_logloss: 0.519114
[148]	valid_0's auc: 0.827097	valid_0's binary_logloss: 0.51872
[149]	valid_0's auc: 0.827645	valid_0's binary_logloss: 0.518309
[150]	valid_0's auc: 0.828087	valid_0's binary_logloss: 0.517926
[151]	valid_0's auc: 0.828442	valid_0's binary_logloss: 0.51762
[152]	valid_0's auc: 0.828774	valid_0's binary_logloss: 0.517237
[153]	valid_0's auc: 0.829262

[71]	valid_0's auc: 0.789365	valid_0's binary_logloss: 0.550857
[72]	valid_0's auc: 0.790102	valid_0's binary_logloss: 0.550346
[73]	valid_0's auc: 0.790492	valid_0's binary_logloss: 0.550013
[74]	valid_0's auc: 0.790938	valid_0's binary_logloss: 0.549647
[75]	valid_0's auc: 0.791511	valid_0's binary_logloss: 0.54922
[76]	valid_0's auc: 0.791993	valid_0's binary_logloss: 0.548846
[77]	valid_0's auc: 0.792522	valid_0's binary_logloss: 0.548391
[78]	valid_0's auc: 0.793038	valid_0's binary_logloss: 0.547987
[79]	valid_0's auc: 0.793691	valid_0's binary_logloss: 0.547495
[80]	valid_0's auc: 0.794366	valid_0's binary_logloss: 0.546971
[81]	valid_0's auc: 0.794805	valid_0's binary_logloss: 0.546609
[82]	valid_0's auc: 0.795258	valid_0's binary_logloss: 0.546226
[83]	valid_0's auc: 0.795688	valid_0's binary_logloss: 0.5459
[84]	valid_0's auc: 0.796274	valid_0's binary_logloss: 0.545452
[85]	valid_0's auc: 0.796997	valid_0's binary_logloss: 0.544893
[86]	valid_0's auc: 0.79756	valid_0's binar

[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[1]	valid_0's auc: 0.73444	valid_0's binary_logloss: 0.671993
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.738729	valid_0's binary_logloss: 0.658116
[3]	valid_0's auc: 0.739896	valid_0's binary_logloss: 0.646683
[4]	valid_0's auc: 0.741722	valid_0's binary_logloss: 0.636989
[5]	valid_0's auc: 0.742268	valid_0's binary_logloss: 0.628868
[6]	valid_0's auc: 0.743936	valid_0's binary_logloss: 0.621913
[7]	valid_0's auc: 0.74455	valid_0's binary_logloss: 0.616099
[8]	valid_0's auc: 0.745144	valid_0's binary_logloss: 0.611036
[9]	valid_0's auc: 0.746164	valid_0's binary_logloss: 0.606684
[10]	valid_0's auc: 0.747025	valid_0's binary_logloss: 0.60292
[11]	valid_0's auc: 0.747714	valid_0's binary_logloss: 0.599651
[12]	valid_0's auc: 0.7

[141]	valid_0's auc: 0.823931	valid_0's binary_logloss: 0.522244
[142]	valid_0's auc: 0.824261	valid_0's binary_logloss: 0.521914
[143]	valid_0's auc: 0.824512	valid_0's binary_logloss: 0.521656
[144]	valid_0's auc: 0.824817	valid_0's binary_logloss: 0.52134
[145]	valid_0's auc: 0.825232	valid_0's binary_logloss: 0.520949
[146]	valid_0's auc: 0.825709	valid_0's binary_logloss: 0.520527
[147]	valid_0's auc: 0.826006	valid_0's binary_logloss: 0.52024
[148]	valid_0's auc: 0.8262	valid_0's binary_logloss: 0.519909
[149]	valid_0's auc: 0.826624	valid_0's binary_logloss: 0.519541
[150]	valid_0's auc: 0.827169	valid_0's binary_logloss: 0.519117
[151]	valid_0's auc: 0.827759	valid_0's binary_logloss: 0.518662
[152]	valid_0's auc: 0.828087	valid_0's binary_logloss: 0.518345
[153]	valid_0's auc: 0.828475	valid_0's binary_logloss: 0.517985
[154]	valid_0's auc: 0.829008	valid_0's binary_logloss: 0.517579
[155]	valid_0's auc: 0.829304	valid_0's binary_logloss: 0.51729
[156]	valid_0's auc: 0.829776	

...

[135]	valid_0's auc: 0.821445	valid_0's binary_logloss: 0.524363
[136]	valid_0's auc: 0.821862	valid_0's binary_logloss: 0.524017
[137]	valid_0's auc: 0.82223	valid_0's binary_logloss: 0.523647
[138]	valid_0's auc: 0.822675	valid_0's binary_logloss: 0.523229
[139]	valid_0's auc: 0.823257	valid_0's binary_logloss: 0.52283
[140]	valid_0's auc: 0.823743	valid_0's binary_logloss: 0.522452
[141]	valid_0's auc: 0.824122	valid_0's binary_logloss: 0.522153
[142]	valid_0's auc: 0.824583	valid_0's binary_logloss: 0.521756
[143]	valid_0's auc: 0.82507	valid_0's binary_logloss: 0.521365
[144]	valid_0's auc: 0.82552	valid_0's binary_logloss: 0.520994
[145]	valid_0's auc: 0.826054	valid_0's binary_logloss: 0.520558
[146]	valid_0's auc: 0.82667	valid_0's binary_logloss: 0.520086
[147]	valid_0's auc: 0.827139	valid_0's binary_logloss: 0.51967
[148]	valid_0's auc: 0.827673	valid_0's binary_logloss: 0.519239
[149]	valid_0's auc: 0.828078	valid_0's binary_logloss: 0.51887
[150]	valid_0's auc: 0.828561	va

[58]	valid_0's auc: 0.781763	valid_0's binary_logloss: 0.557423
[59]	valid_0's auc: 0.782316	valid_0's binary_logloss: 0.556944
[60]	valid_0's auc: 0.783005	valid_0's binary_logloss: 0.556421
[61]	valid_0's auc: 0.783661	valid_0's binary_logloss: 0.555932
[62]	valid_0's auc: 0.78432	valid_0's binary_logloss: 0.55544
[63]	valid_0's auc: 0.784756	valid_0's binary_logloss: 0.555072
[64]	valid_0's auc: 0.785411	valid_0's binary_logloss: 0.554544
[65]	valid_0's auc: 0.786165	valid_0's binary_logloss: 0.554024
[66]	valid_0's auc: 0.786768	valid_0's binary_logloss: 0.55351
[67]	valid_0's auc: 0.787273	valid_0's binary_logloss: 0.553036
[68]	valid_0's auc: 0.787845	valid_0's binary_logloss: 0.552601
[69]	valid_0's auc: 0.788142	valid_0's binary_logloss: 0.552132
[70]	valid_0's auc: 0.788702	valid_0's binary_logloss: 0.551736
[71]	valid_0's auc: 0.789198	valid_0's binary_logloss: 0.551318
[72]	valid_0's auc: 0.789667	valid_0's binary_logloss: 0.550947
[73]	valid_0's auc: 0.790242	valid_0's bina

[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
[1]	valid_0's auc: 0.73444	valid_0's binary_logloss: 0.671993
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.738709	valid_0's binary_logloss: 0.658121
[3]	valid_0's auc: 0.739853	valid_0's binary_logloss: 0.646689
[4]	valid_0's auc: 0.741675	valid_0's binary_logloss: 0.636995
[5]	valid_0's auc: 0.74221	valid_0's binary_logloss: 0.628898
[6]	valid_0's auc: 0.743858	valid_0's binary_logloss: 0.621946
[7]	valid_0's auc: 0.744477	valid_0's binary_logloss: 0.616136
[8]	valid_0's auc: 0.745002	valid_0's binary_logloss: 0.611088
[9]	valid_0's auc: 0.746041	valid_0's binary_logloss: 0.606733
[10]	valid_0's auc: 0.746896	valid_0's binary_logloss: 0.602969
[11]	valid_0's auc: 0.747617	valid_0's binary_logloss: 0.599693
[12]	valid_0's auc: 0.

[125]	valid_0's auc: 0.816939	valid_0's binary_logloss: 0.528796
[126]	valid_0's auc: 0.817187	valid_0's binary_logloss: 0.528555
[127]	valid_0's auc: 0.817735	valid_0's binary_logloss: 0.528138
[128]	valid_0's auc: 0.818141	valid_0's binary_logloss: 0.527683
[129]	valid_0's auc: 0.818731	valid_0's binary_logloss: 0.527251
[130]	valid_0's auc: 0.819221	valid_0's binary_logloss: 0.526852
[131]	valid_0's auc: 0.819737	valid_0's binary_logloss: 0.526419
[132]	valid_0's auc: 0.820328	valid_0's binary_logloss: 0.525938
[133]	valid_0's auc: 0.820744	valid_0's binary_logloss: 0.525557
[134]	valid_0's auc: 0.821191	valid_0's binary_logloss: 0.525174
[135]	valid_0's auc: 0.821473	valid_0's binary_logloss: 0.524851
[136]	valid_0's auc: 0.822048	valid_0's binary_logloss: 0.524419
[137]	valid_0's auc: 0.822577	valid_0's binary_logloss: 0.523998
[138]	valid_0's auc: 0.823058	valid_0's binary_logloss: 0.523609
[139]	valid_0's auc: 0.82349	valid_0's binary_logloss: 0.523279
[140]	valid_0's auc: 0.823

[184]	training's auc: 0.858033	training's binary_logloss: 0.495277
[185]	training's auc: 0.85856	training's binary_logloss: 0.494908
[186]	training's auc: 0.859015	training's binary_logloss: 0.49453
[187]	training's auc: 0.859509	training's binary_logloss: 0.494146
[188]	training's auc: 0.859931	training's binary_logloss: 0.493716
[189]	training's auc: 0.860278	training's binary_logloss: 0.493405
[190]	training's auc: 0.860653	training's binary_logloss: 0.493036
[191]	training's auc: 0.861098	training's binary_logloss: 0.492613
[192]	training's auc: 0.861479	training's binary_logloss: 0.492203
[193]	training's auc: 0.861888	training's binary_logloss: 0.491817
[194]	training's auc: 0.862229	training's binary_logloss: 0.491504
[195]	training's auc: 0.862527	training's binary_logloss: 0.491208
[196]	training's auc: 0.863117	training's binary_logloss: 0.490742
[197]	training's auc: 0.863524	training's binary_logloss: 0.49039
[198]	training's auc: 0.863979	training's binary_logloss: 0.49001

In [24]:
lgbm_clf=lgbm.LGBMClassifier(n_estimators=500, max_depth=64, min_data_in_leaf=15)

In [25]:

Stack_final_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1)
Stack_final_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis=1)
#print('원본 학습 피처 데이터 Shape:',X_train.shape, '원본 테스트 피처 Shape:',X_test.shape)
#print('스태킹 학습 피처 데이터 Shape:', Stack_final_X_train.shape,
  #    '스태킹 테스트 피처 데이터 Shape:',Stack_final_X_test.shape)

In [26]:
lgbm_clf.fit(Stack_final_X_train, train_y)

LGBMClassifier(max_depth=64, min_data_in_leaf=15, n_estimators=500)

In [27]:
pred_y = lgbm_clf.predict(Stack_final_X_test)
pred_y

array([2, 2, 1, ..., 1, 2, 2], dtype=int64)

In [28]:
submission['voted']=pred_y
submission

,voted
index,
0,2
1,2
2,1
3,1
4,2
...,...
11378,2
11379,2
11380,1


In [29]:
submission.to_csv('submission3.csv')